In [ ]:
import pandas as pd

# Read the Excel file
file_path = 'dynamic_uncertainty_2022—2050.xlsx'
excel_data = pd.ExcelFile(file_path)

# Get all sheet names
sheet_names = excel_data.sheet_names

# Read all sheets
sheets = {sheet_name: excel_data.parse(sheet_name) for sheet_name in sheet_names}

# Print all sheet names
print(sheet_names)

['估计参数', '筛选178国', '估计参数c', '估计参数s', '输出结果数据排列', 'Base_ CDD 18.3C + HWD 95P', 'CDD 18.3C + HWD 99P', 'CDD 18.3C + HWD 90P', 'adj.HWD 95P + CDD 18.3C', 'CDD 24C + HWD 95P']


In [4]:
df_bs = sheets['Base_ CDD 18.3C + HWD 95P']
df_A = sheets['CDD 18.3C + HWD 99P']
df_B = sheets['CDD 18.3C + HWD 90P']
df_C = sheets['adj.HWD 95P + CDD 18.3C']
df_D = sheets['CDD 24C + HWD 95P']
para_c = sheets['估计参数c']
para_s = sheets['估计参数s']
list = sheets['筛选178国']
list_filtered = list[['country.2', '穷富']]
df_bs_filtered = df_bs.merge(list_filtered, left_on='Country', right_on='country.2', how='inner')
df_A_filtered = df_A.merge(list_filtered, left_on='Country', right_on='country.2', how='inner')
df_B_filtered = df_B.merge(list_filtered, left_on='Country', right_on='country.2', how='inner')
df_C_filtered = df_C.merge(list_filtered, left_on='Country', right_on='country.2', how='inner')
df_D_filtered = df_D.merge(list_filtered, left_on='Country', right_on='country.2', how='inner')
import numpy as np
df_0_op = df_bs_filtered.copy()
df_1_op = df_A_filtered.copy()
df_2_op = df_B_filtered.copy()
df_3_op = df_C_filtered.copy()
df_4_op = df_D_filtered.copy()
for i in range(5):
    exec('df_%d_op = df_bs_filtered.copy()' % (i+5))
for i in range(10):
    exec('df_%d_op.replace("-", np.nan, inplace=True)' % i)

## Compute by Country

In [ ]:
## Compute COUNTRY
import os
import pandas as pd
if not os.path.exists('All_By_Country.xlsx'):
    # Create the file with an empty DataFrame
    with pd.ExcelWriter('All_By_Country.xlsx') as writer:
        pd.DataFrame().to_excel(writer, sheet_name='Sheet1')
if not os.path.exists('Result_AVG_By_Country.xlsx'):
    # Create the file with an empty DataFrame
    with pd.ExcelWriter('Result_AVG_By_Country.xlsx') as writer:
        pd.DataFrame().to_excel(writer, sheet_name='Sheet1')
if not os.path.exists('Result_MED_By_Country.xlsx'):
    # Create the file with an empty DataFrame
    with pd.ExcelWriter('Result_MED_By_Country.xlsx') as writer:
        pd.DataFrame().to_excel(writer, sheet_name='Sheet1')
li = ['Baseline','Heatwave 99p','Heatwave 90p','CDD-unrelated Heatwave',
      'CDD 24℃','NO CHN/USA','NO EU','NO Yr × Poor FE','NO RTOs × Yr FE','Before COVID-19']
colli = ['SSP119: Avg', 'SSP119: Max', 'SSP119: Min', 'SSP126: Avg',
      'SSP126: Max', 'SSP126: Min', 'SSP534: Avg', 'SSP534: Max',
      'SSP534: Min', 'SSP585: Avg', 'SSP585: Max', 'SSP585: Min']
sli = ['SSP119','SSP126','SSP534','SSP585']

res_c_a_all = []
res_c_m_all = []

for i in range(len(li)):
    exec(f"sc_c_{i} = pd.DataFrame()")
    sc_c_a = eval(f"df_{i}_op.iloc[:, 2:14] * para_c.iloc[2, {i}]") 
    sc_c_b = eval(f"df_{i}_op.iloc[:, 14:26] * para_c.iloc[0, {i}]")
    A_values = eval(f"df_{i}_op.iloc[:, 2:14].to_numpy()")
    B_values = eval(f"df_{i}_op.iloc[:, 14:26].to_numpy()")
    sc_c_c = A_values * B_values * para_c.iloc[1, i]
    exec(f"sc_c_{i}[colli] = sc_c_a.to_numpy() + sc_c_b.to_numpy() + sc_c_c")

    # Organize merged results of all data tables
    all_c = []
    for s in range(len(sli)):
        exec(f"sc_c_{i}_{sli[s]} = pd.DataFrame()")
        exec(f"sc_c_{i}_{sli[s]}[['Avg','Max','Min']] = sc_c_{i}.iloc[:,s*3:(s+1)*3]")
        exec(f"sc_c_{i}_{sli[s]}['Country'] = df_{i}_op['Country']")
        exec(f"sc_c_{i}_{sli[s]}['穷富'] = df_{i}_op['穷富']")
        exec(f"sc_c_{i}_{sli[s]}['Year'] = df_{i}_op['Year']")
        exec(f"sc_c_{i}_{sli[s]}[['AvgHWD','MaxHWD','MinHWD']] = df_{i}_op.iloc[:,(2+s*3):(2+(s+1)*3)]")
        exec(f"sc_c_{i}_{sli[s]}['SSP'] = sli[{s}]")
    
        exec(f"all_c.append(sc_c_{i}_{sli[s]})")

    all_c_df = pd.concat(all_c, ignore_index=True)
    # Group and compute mean and median
    filtered_df_Avg = all_c_df
    filtered_df_Max = all_c_df
    filtered_df_Min = all_c_df
    # filtered_df_Avg = all_c_df[(all_c_df['穷富'] == 'Rich') & (all_c_df['AvgHWD'] > 4.1)]
    # filtered_df_Max = all_c_df[(all_c_df['穷富'] == 'Rich') & (all_c_df['MaxHWD'] > 4.1)]
    # filtered_df_Min = all_c_df[(all_c_df['穷富'] == 'Rich') & (all_c_df['MinHWD'] > 4.1)]
    res_c_Avg_a = filtered_df_Avg.groupby(['SSP', 'Year']).agg(
        Avg_Avg=('Avg', 'mean'),
    ).reset_index()
    res_c_Max_a = filtered_df_Max.groupby(['SSP', 'Year']).agg(
        Avg_Max=('Max', 'mean'),
    ).reset_index()
    res_c_Min_a = filtered_df_Min.groupby(['SSP', 'Year']).agg(
        Avg_Min=('Min', 'mean'),
    ).reset_index()
    res_c_a = res_c_Avg_a.merge(res_c_Max_a, on=['SSP', 'Year'], how='left').merge(res_c_Min_a, on=['SSP', 'Year'], how='left')
    
    res_c_Avg_m = filtered_df_Avg.groupby(['SSP', 'Year']).agg(
        Median_Avg=('Avg', 'median'),
    ).reset_index()
    res_c_Max_m = filtered_df_Max.groupby(['SSP', 'Year']).agg(
        Median_Max=('Max', 'median'),
    ).reset_index()
    res_c_Min_m = filtered_df_Min.groupby(['SSP', 'Year']).agg(
        Median_Min=('Min', 'median'),
    ).reset_index()
    res_c_m = res_c_Avg_m.merge(res_c_Max_m, on=['SSP', 'Year'], how='left').merge(res_c_Min_m, on=['SSP', 'Year'], how='left')
    # Save results

    res_c_a['Robust'] = li[i]
    res_c_m['Robust'] = li[i]
    res_c_a_all.append(res_c_a)
    res_c_m_all.append(res_c_m)

final_df_c_a = pd.concat(res_c_a_all, axis=0)
final_df_c_m = pd.concat(res_c_m_all, axis=0)
final_df_c_a = final_df_c_a.reset_index(drop=True)
final_df_c_m = final_df_c_m.reset_index(drop=True)
final_df_c_a.to_excel('./final/All_Result_AVG_by_Country.xlsx')
final_df_c_m.to_excel('./final/All_Result_MED_by_Country.xlsx')



## Compute by Sector

In [ ]:
# Final version
import os
final_s_a = []
final_s_m = []

li = ['Baseline','Heatwave 99p','Heatwave 90p','CDD-unrelated Heatwave',
      'CDD 24℃','NO CHN/USA','NO EU','NO Yr × Poor FE','NO RTOs × Yr FE','Before COVID-19']
for dat in range(len(li)):
    name = li[dat].replace(' ', '_').replace('/', '_').replace('×', 'x').replace('℃', 'C')
    f1 = 'All_By_Sector_' + name + '.xlsx'
    f2 = 'Result_AVG_By_Sector_' + name + '.xlsx'
    f3 = 'Result_MED_By_Sector_' + name + '.xlsx'
    if not os.path.exists(f1):
        # Create the file with an empty DataFrame
        with pd.ExcelWriter(f1) as writer:
            pd.DataFrame().to_excel(writer, sheet_name='Sheet1')
    if not os.path.exists(f2):
        # Create the file with an empty DataFrame
        with pd.ExcelWriter(f2) as writer:
            pd.DataFrame().to_excel(writer, sheet_name='Sheet1')
    if not os.path.exists(f3):
        # Create the file with an empty DataFrame
        with pd.ExcelWriter(f3) as writer:
            pd.DataFrame().to_excel(writer, sheet_name='Sheet1')

    se = para_s.iloc[:,0].tolist()

    colli = ['SSP119: Avg', 'SSP119: Max', 'SSP119: Min', 'SSP126: Avg',
        'SSP126: Max', 'SSP126: Min', 'SSP534: Avg', 'SSP534: Max',
        'SSP534: Min', 'SSP585: Avg', 'SSP585: Max', 'SSP585: Min']
    sli = ['SSP119','SSP126','SSP534','SSP585']


    exec(f"df_op = df_{dat}_op")
    res_s_a_all = []
    res_s_m_all = []
    for i in range(len(se)):
        exec(f"sc_s_{i} = pd.DataFrame()")
        sc_s_a = eval(f"df_op.iloc[:, 2:14] * para_s.iloc[{i},{dat}*3+4]")  # HWD
        sc_s_b = eval(f"df_op.iloc[:, 14:26] * para_s.iloc[{i},{dat}*3+2]") # CDD
        A_values = eval(f"df_op.iloc[:, 2:14].to_numpy()")
        B_values = eval(f"df_op.iloc[:, 14:26].to_numpy()")
        exec(f"sc_s_c = A_values * B_values * para_s.iloc[i,{dat}*3+3]")
        exec(f"sc_s_{i}[colli] = sc_s_a.to_numpy() + sc_s_b.to_numpy() + sc_s_c") # Sum for each sector by country

        # Organize merged results of all data tables
        all_s = []
        for s in range(len(sli)):
            exec(f"sc_s_{i}_{sli[s]} = pd.DataFrame()")
            exec(f"sc_s_{i}_{sli[s]}[['Avg','Max','Min']] = sc_s_{i}.iloc[:,s*3:(s+1)*3]")
            exec(f"sc_s_{i}_{sli[s]}['Country'] = df_op['Country']")
            exec(f"sc_s_{i}_{sli[s]}['穷富'] = df_op['穷富']")
            exec(f"sc_s_{i}_{sli[s]}['Year'] = df_op['Year']")
            exec(f"sc_s_{i}_{sli[s]}[['AvgHWD','MaxHWD','MinHWD']] = df_op.iloc[:,(2+s*3):(2+(s+1)*3)]")
            exec(f"sc_s_{i}_{sli[s]}['SSP'] = sli[{s}]")
        
            exec(f"all_s.append(sc_s_{i}_{sli[s]})")
        
        # Save results
        all_s_df = pd.concat(all_s, ignore_index=True)
        if not np.isnan(para_s.iloc[i,1]):
            filtered_df_Avg = all_s_df[all_s_df['AvgHWD'] > para_s.iloc[i,1]]
            filtered_df_Max = all_s_df[all_s_df['MaxHWD'] > para_s.iloc[i,1]]
            filtered_df_Min = all_s_df[all_s_df['MinHWD'] > para_s.iloc[i,1]]
        else:
            filtered_df_Avg = all_s_df
            filtered_df_Max = all_s_df
            filtered_df_Min = all_s_df
        # Group and compute mean and median
        res_s_Avg_a = filtered_df_Avg.groupby(['SSP', 'Year']).agg(
            Avg_Avg=('Avg', 'mean'),
        ).reset_index()
        res_s_Max_a = filtered_df_Max.groupby(['SSP', 'Year']).agg(
            Avg_Max=('Max', 'mean'),
        ).reset_index()
        res_s_Min_a = filtered_df_Min.groupby(['SSP', 'Year']).agg(
            Avg_Min=('Min', 'mean'),
        ).reset_index()
        res_s_a = res_s_Avg_a.merge(res_s_Max_a, on=['SSP', 'Year'], how='left').merge(res_s_Min_a, on=['SSP', 'Year'], how='left')

        res_s_Avg_m = filtered_df_Avg.groupby(['SSP', 'Year']).agg(
            Median_Avg=('Avg', 'median'),
        ).reset_index()
        res_s_Max_m = filtered_df_Max.groupby(['SSP', 'Year']).agg(
            Median_Max=('Max', 'median'),
        ).reset_index()
        res_s_Min_m = filtered_df_Min.groupby(['SSP', 'Year']).agg(
            Median_Min=('Min', 'median'),
        ).reset_index()
        res_s_m = res_s_Avg_m.merge(res_s_Max_m, on=['SSP', 'Year'], how='left').merge(res_s_Min_m, on=['SSP', 'Year'], how='left')
        
        res_s_a['Sector'] = se[i]
        res_s_m['Sector'] = se[i]
        res_s_a_all.append(res_s_a)
        res_s_m_all.append(res_s_m) 
    final_df_s_a = pd.concat(res_s_a_all, axis=0)
    final_df_s_m = pd.concat(res_s_m_all, axis=0)
    final_df_s_a = final_df_s_a.reset_index(drop=True)
    final_df_s_m = final_df_s_m.reset_index(drop=True)
    final_df_s_a['Robust'] = li[dat]
    final_df_s_m['Robust'] = li[dat]
    final_s_a.append(final_df_s_a)
    final_s_m.append(final_df_s_m)
    
final_s_a_a = pd.concat(final_s_a, axis=0)
final_s_m_a = pd.concat(final_s_m, axis=0)
final_s_a_a.to_excel('./final/All_Result_AVG_by_Sector.xlsx')
final_s_m_a.to_excel('./final/All_Result_MED_by_Sector.xlsx')


In [ ]:
import pandas as pd

# Read tables A and B
file_path_A = 'All_Result_MED_by_Sector_NoFilt.xlsx'
file_path_B = 'All_Result_MED_by_Sector_Filt.xlsx'

# Assume the sheet name in both A and B is 'sheet1'
sheet_name = 'Sheet1'

# Read the Excel files
df_A = pd.read_excel(file_path_A, sheet_name=sheet_name)
df_B = pd.read_excel(file_path_B, sheet_name=sheet_name)

# Left join by "SSP", "Year", "Sector", and "Robust"
merged_df = pd.merge(df_A, df_B, on=["SSP", "Year", "Sector", "Robust"], how="left")

# Save the merged data to a new Excel file
output_file_path = 'merged_output_MED.xlsx'
merged_df.to_excel(output_file_path, index=False)